# Strategy 0.4 

Merupakan ekstensi dari strategy 0.3. Namun dilakukan ekstraksi gelar yang didapatkan dari column Name.

In [1]:
import sys
sys.path.append('..')
from src.func import custom_info

import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv("../data/interim/for-strategy4.csv", index_col="PassengerId")

X = df.drop(columns="Survived")
y = df.Survived
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

numerical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", MinMaxScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder())
])

numerical_columns = ["SibSp", "Parch"]
categorical_columns = ["Pclass", "Sex", "Age", "Fare", "Embarked", "isAlone", "Title"]

preprocessor = ColumnTransformer([
    ("numeric", numerical_pipeline, numerical_columns),
    ("categoric", categorical_pipeline, categorical_columns)
])

pipeline = Pipeline([
    ("prep", preprocessor),
    ("knn", KNeighborsClassifier())
])

parameter = {
    "knn__n_neighbors" : range(1,31,2),
    "knn__weights" : ["uniform", "distance"],
    "knn__p": [1, 1.5, 2]
}

model = GridSearchCV(pipeline, parameter, cv=3, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

Fitting 3 folds for each of 90 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:    7.6s finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['SibSp',
                                                                          'Parch']),
                                                                        ('categoric',
                                                                         Pipeline(steps=[('imputer',
                                                                            

In [3]:
model.best_params_

{'knn__n_neighbors': 11, 'knn__p': 1.5, 'knn__weights': 'uniform'}

In [4]:
model.score(X_train, y_train), model.score(X_test, y_test)

(0.8455056179775281, 0.8044692737430168)